# **Hackathon**

## **Getting Data**

**Installing kaggle dependencies**

In [1]:
!pip install kaggle
!mkdir .kaggle

**Importing data**

In [3]:
import json
token = {"username":"prakritisahu","key":"c8dce2c6b383533a84d1ddc341353420"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

!kaggle config set -n path -v{/content}

!chmod 600 /root/.kaggle/kaggle.json

!kaggle competitions download -c Foml-2021

- path is now set to: {/content}
  0% 0.00/4.80M [00:00<?, ?B/s]
100% 4.80M/4.80M [00:00<00:00, 44.2MB/s]
 72% 5.00M/6.97M [00:00<00:00, 52.1MB/s]
100% 6.97M/6.97M [00:00<00:00, 63.8MB/s]


In [4]:
!unzip /content/{/content}/competitions/Foml-2021/train.csv.zip && rm /content/{/content}/competitions/Foml-2021/train.csv.zip

Archive:  /content/{/content}/competitions/Foml-2021/train.csv.zip
  inflating: train.csv               


In [5]:
!unzip /content/{/content}/competitions/Foml-2021/test.csv.zip && rm /content/{/content}/competitions/Foml-2021/test.csv.zip

Archive:  /content/{/content}/competitions/Foml-2021/test.csv.zip
  inflating: test.csv                


**Importing necessary libraries**

In [6]:
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import math
import sklearn.ensemble
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

**Reading data**

In [7]:
train_data=pd.read_csv("train.csv")
test_data=pd.read_csv("test.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
train_data.describe()

,Speed Limit,Vehicle Year,Latitude,Longitude,Fault
count,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000
mean,32.895319,1974.545465,39.082783,-77.112688,0.550204
std,11.696668,483.279428,0.072062,0.098300,0.497478
min,0.000000,0.000000,38.008120,-79.486000,0.000000
25%,25.000000,2005.000000,39.023974,-77.188926,0.000000
50%,35.000000,2010.000000,39.074522,-77.105670,1.000000
75%,40.000000,2014.000000,39.139685,-77.040114,1.000000
max,70.000000,9999.000000,39.989747,-75.975952,1.000000


In [9]:
train_data.shape

(51490, 42)

## **Data Preprocessing and Feature Selection**

In [10]:
train_data.columns

Index(['Report Number', 'Local Case Number', 'Agency Name', 'ACRS Report Type',
       'Crash Date/Time', 'Route Type', 'Road Name', 'Cross-Street Type',
       'Cross-Street Name', 'Off-Road Description', 'Municipality',
       'Related Non-Motorist', 'Collision Type', 'Weather',
       'Surface Condition', 'Light', 'Traffic Control',
       'Driver Substance Abuse', 'Non-Motorist Substance Abuse', 'Person ID',
       'Injury Severity', 'Circumstance', 'Drivers License State',
       'Vehicle ID', 'Vehicle Damage Extent', 'Vehicle First Impact Location',
       'Vehicle Second Impact Location', 'Vehicle Body Type',
       'Vehicle Movement', 'Vehicle Continuing Dir', 'Vehicle Going Dir',
       'Speed Limit', 'Driverless Vehicle', 'Parked Vehicle', 'Vehicle Year',
       'Vehicle Make', 'Vehicle Model', 'Equipment Problems', 'Latitude',
       'Longitude', 'Location', 'Fault'],
      dtype='object')

In [11]:
train_data.columns = train_data.columns.str.replace(' ', '_')
train_data.columns = train_data.columns.str.replace('/', '_')
train_data.columns = train_data.columns.str.replace('-', '_')
train_data.columns

Index(['Report_Number', 'Local_Case_Number', 'Agency_Name', 'ACRS_Report_Type',
       'Crash_Date_Time', 'Route_Type', 'Road_Name', 'Cross_Street_Type',
       'Cross_Street_Name', 'Off_Road_Description', 'Municipality',
       'Related_Non_Motorist', 'Collision_Type', 'Weather',
       'Surface_Condition', 'Light', 'Traffic_Control',
       'Driver_Substance_Abuse', 'Non_Motorist_Substance_Abuse', 'Person_ID',
       'Injury_Severity', 'Circumstance', 'Drivers_License_State',
       'Vehicle_ID', 'Vehicle_Damage_Extent', 'Vehicle_First_Impact_Location',
       'Vehicle_Second_Impact_Location', 'Vehicle_Body_Type',
       'Vehicle_Movement', 'Vehicle_Continuing_Dir', 'Vehicle_Going_Dir',
       'Speed_Limit', 'Driverless_Vehicle', 'Parked_Vehicle', 'Vehicle_Year',
       'Vehicle_Make', 'Vehicle_Model', 'Equipment_Problems', 'Latitude',
       'Longitude', 'Location', 'Fault'],
      dtype='object')

In [12]:
train_data.Vehicle_Year.unique()

array([2005, 2018, 2016, 2001, 2013, 2015, 2004, 2014, 2003, 2012, 2006,
       2002, 1997,    0, 2007, 2011, 2017, 2009, 1982, 2010, 2008, 2019,
       2020, 1991, 2040, 1988, 1955,  200, 2000, 1972,    2, 1999,   97,
       1998, 1900, 2996, 1005, 1986, 1985, 1990, 1995, 3003, 1994, 2021,
       1992, 1993, 2914, 2033, 1996, 1987, 9999, 1969,   15, 1970, 1025,
       8008, 2991, 1965, 1979, 1974, 2911, 1946,   13,    1, 1963, 2103,
       1980, 8888,   99, 1978, 2099, 1111, 1983,   14, 7817, 2107, 1968,
       1989, 1984, 1901, 2022,  215, 1966])

We can see a lot of nonsensical values for the 'vehicle year column', so we replace them.

In [13]:
train_data['Vehicle_Year'] = train_data['Vehicle_Year'].replace([0,2040,200,2,97,2996,1005,3003,2914,2033,9999,15,1025,8008,2991,2911,13,1,2103,8888,99,2099,1111,14,7817,2107,2022,215]
                                                            , [2000,0,2000,0,1997,1996,2005,2003,2014,0,1999,2015,0,2008,0,2011,2013,0,0,0,1999,0,0,2014,0,0,0,2015])
train_data['Vehicle_Year']=train_data['Vehicle_Year'].fillna(0)

In [14]:
train_data['report_id'] = train_data['Report_Number'].apply(lambda x: x[0:2])
train_data.report_id.unique()

array(['MP', 'D5', 'E7', 'D8', 'H2', 'H1'], dtype=object)

In [15]:
train_data = train_data.drop(['Report_Number', 'Person_ID', 'Vehicle_ID', 'Driverless_Vehicle', 
                              'Vehicle_Make', 'Vehicle_Model', 'Road_Name', 'Cross_Street_Name' ], axis=1)
#'Vehicle_Year', 'Local_Case_Number', 'Agency_Name', 'Municipality',
train_data.shape

(51490, 35)

In [16]:
train_data['Crash_Date_Time'] =  pd.to_datetime(train_data['Crash_Date_Time'])
train_data['crash_year'] = train_data['Crash_Date_Time'].dt.year
train_data['crash_quarter'] = train_data['Crash_Date_Time'].dt.quarter
train_data['crash_month'] = train_data['Crash_Date_Time'].dt.month
train_data['crash_day'] = train_data['Crash_Date_Time'].dt.day
train_data['crash_hour'] = train_data['Crash_Date_Time'].dt.hour

In [17]:
train_data=train_data.drop(['Crash_Date_Time'],axis=1)

In [18]:
train_data.head(2)

,Local_Case_Number,Agency_Name,ACRS_Report_Type,Route_Type,Cross_Street_Type,Off_Road_Description,Municipality,Related_Non_Motorist,Collision_Type,Weather,Surface_Condition,Light,Traffic_Control,Driver_Substance_Abuse,Non_Motorist_Substance_Abuse,Injury_Severity,Circumstance,Drivers_License_State,Vehicle_Damage_Extent,Vehicle_First_Impact_Location,Vehicle_Second_Impact_Location,Vehicle_Body_Type,Vehicle_Movement,Vehicle_Continuing_Dir,Vehicle_Going_Dir,Speed_Limit,Parked_Vehicle,Vehicle_Year,Equipment_Problems,Latitude,Longitude,Location,Fault,report_id,crash_year,crash_quarter,crash_month,crash_day,crash_hour
0,10196,Montgomery County Police,Injury Crash,Maryland (State),County,12305 SHOREFIELD RD,NaN,NaN,SINGLE VEHICLE,CLEAR,WET,DAYLIGHT,NO CONTROLS,NONE DETECTED,NaN,SUSPECTED MINOR INJURY,"N/A, WET",MD,DESTROYED,TWELVE OCLOCK,TWELVE OCLOCK,PICKUP TRUCK,MOVING CONSTANT SPEED,South,Unknown,40,No,2005,NO MISUSE,39.263378,-77.344203,3.6383 7.4233,1,MP,2017,2,6,5,16
1,20080,Montgomery County Police,Injury Crash,County,County,NaN,NaN,NaN,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,STOP SIGN,NONE DETECTED,NaN,NO APPARENT INJURY,"BACKUP DUE TO REGULAR CONGESTION, SLEET, HAIL,...",MD,DISABLING,TWELVE OCLOCK,TWELVE OCLOCK,PASSENGER CAR,ACCELERATING,South,South,25,Yes,2018,NO MISUSE,39.068913,-77.063227,3.0862 7.324,1,MP,2020,1,2,22,10


In [19]:
print("Total null values:",train_data.isnull().sum().sum())

Total null values: 285641


In [20]:
def NaNcheck(data):
  columns=data.columns 
  nan_count=[]
  for i in columns:
    nan_count.append(data[i].isnull().sum())
  arr=np.array(nan_count)
  
  index_values=np.where(arr>=20000)
  print(index_values,np.count_nonzero(index_values))

  return index_values,arr

index_values,arr=NaNcheck(train_data)

columns=train_data.columns
columns_delete=columns[index_values]

print(f"Columns with NaN values greater than 40,000:{columns_delete} ")


(array([ 5,  6,  7, 14, 16]),) 5
Columns with NaN values greater than 40,000:Index(['Off_Road_Description', 'Municipality', 'Related_Non_Motorist',
       'Non_Motorist_Substance_Abuse', 'Circumstance'],
      dtype='object') 


Since the columns with a large number of NaN values are important for prediction and cannot be deleted, so we instead encode the NaN values

**Converting categorical data**

In [21]:
l=[]

for i in train_data.columns:
   if train_data[i].dtypes == "object":
     l.append(i)
print(l)
print(len(l))

['Local_Case_Number', 'Agency_Name', 'ACRS_Report_Type', 'Route_Type', 'Cross_Street_Type', 'Off_Road_Description', 'Municipality', 'Related_Non_Motorist', 'Collision_Type', 'Weather', 'Surface_Condition', 'Light', 'Traffic_Control', 'Driver_Substance_Abuse', 'Non_Motorist_Substance_Abuse', 'Injury_Severity', 'Circumstance', 'Drivers_License_State', 'Vehicle_Damage_Extent', 'Vehicle_First_Impact_Location', 'Vehicle_Second_Impact_Location', 'Vehicle_Body_Type', 'Vehicle_Movement', 'Vehicle_Continuing_Dir', 'Vehicle_Going_Dir', 'Parked_Vehicle', 'Equipment_Problems', 'Location', 'report_id']
29


In [22]:
dt=train_data
le = LabelEncoder()

for j in l:
  dt[j]=dt[j].astype('|S')

for i in l:
  dt[i]=le.fit_transform(dt[i])

dt[l].head()

,Local_Case_Number,Agency_Name,ACRS_Report_Type,Route_Type,Cross_Street_Type,Off_Road_Description,Municipality,Related_Non_Motorist,Collision_Type,Weather,Surface_Condition,Light,Traffic_Control,Driver_Substance_Abuse,Non_Motorist_Substance_Abuse,Injury_Severity,Circumstance,Drivers_License_State,Vehicle_Damage_Extent,Vehicle_First_Impact_Location,Vehicle_Second_Impact_Location,Vehicle_Body_Type,Vehicle_Movement,Vehicle_Continuing_Dir,Vehicle_Going_Dir,Parked_Vehicle,Equipment_Problems,Location,report_id
0,216,5,1,3,0,161,20,11,15,2,10,4,1,8,9,3,98,25,0,12,12,19,9,2,3,0,4,23738,5
1,928,5,1,0,0,3935,20,11,16,2,0,4,6,8,9,1,30,25,1,12,12,18,0,2,2,1,4,3146,5
2,114,5,2,3,3,3935,20,11,13,2,0,7,2,11,9,1,139,10,5,9,9,26,13,4,4,0,4,36906,5
3,229,5,1,0,0,3935,20,11,7,3,0,4,1,8,9,2,139,25,2,12,13,18,9,2,2,0,5,31353,5
4,795,5,1,3,0,3935,20,11,9,2,0,4,7,8,4,3,139,25,1,9,9,18,17,2,2,0,4,38537,5


In [23]:
train_data=dt
train_data.shape

(51490, 39)

Scaling Features

In [24]:
train_data.columns

Index(['Local_Case_Number', 'Agency_Name', 'ACRS_Report_Type', 'Route_Type',
       'Cross_Street_Type', 'Off_Road_Description', 'Municipality',
       'Related_Non_Motorist', 'Collision_Type', 'Weather',
       'Surface_Condition', 'Light', 'Traffic_Control',
       'Driver_Substance_Abuse', 'Non_Motorist_Substance_Abuse',
       'Injury_Severity', 'Circumstance', 'Drivers_License_State',
       'Vehicle_Damage_Extent', 'Vehicle_First_Impact_Location',
       'Vehicle_Second_Impact_Location', 'Vehicle_Body_Type',
       'Vehicle_Movement', 'Vehicle_Continuing_Dir', 'Vehicle_Going_Dir',
       'Speed_Limit', 'Parked_Vehicle', 'Vehicle_Year', 'Equipment_Problems',
       'Latitude', 'Longitude', 'Location', 'Fault', 'report_id', 'crash_year',
       'crash_quarter', 'crash_month', 'crash_day', 'crash_hour'],
      dtype='object')

In [25]:
col=['Local_Case_Number', 'Agency_Name', 'ACRS_Report_Type', 'Route_Type',
       'Cross_Street_Type', 'Off_Road_Description', 'Municipality',
       'Related_Non_Motorist', 'Collision_Type', 'Weather',
       'Surface_Condition', 'Light', 'Traffic_Control',
       'Driver_Substance_Abuse', 'Non_Motorist_Substance_Abuse',
       'Injury_Severity', 'Circumstance', 'Drivers_License_State',
       'Vehicle_Damage_Extent', 'Vehicle_First_Impact_Location',
       'Vehicle_Second_Impact_Location', 'Vehicle_Body_Type',
       'Vehicle_Movement', 'Vehicle_Continuing_Dir', 'Vehicle_Going_Dir',
       'Speed_Limit', 'Parked_Vehicle', 'Vehicle_Year', 'Equipment_Problems',
       'Latitude', 'Longitude', 'Location', 'report_id', 'crash_year',
       'crash_quarter', 'crash_month', 'crash_day', 'crash_hour']

scaler=StandardScaler().fit(train_data[col])
train_scaled_data=scaler.transform(train_data[col])
train_data.head()

,Local_Case_Number,Agency_Name,ACRS_Report_Type,Route_Type,Cross_Street_Type,Off_Road_Description,Municipality,Related_Non_Motorist,Collision_Type,Weather,Surface_Condition,Light,Traffic_Control,Driver_Substance_Abuse,Non_Motorist_Substance_Abuse,Injury_Severity,Circumstance,Drivers_License_State,Vehicle_Damage_Extent,Vehicle_First_Impact_Location,Vehicle_Second_Impact_Location,Vehicle_Body_Type,Vehicle_Movement,Vehicle_Continuing_Dir,Vehicle_Going_Dir,Speed_Limit,Parked_Vehicle,Vehicle_Year,Equipment_Problems,Latitude,Longitude,Location,Fault,report_id,crash_year,crash_quarter,crash_month,crash_day,crash_hour
0,216,5,1,3,0,161,20,11,15,2,10,4,1,8,9,3,98,25,0,12,12,19,9,2,3,40,0,2005,4,39.263378,-77.344203,23738,1,5,2017,2,6,5,16
1,928,5,1,0,0,3935,20,11,16,2,0,4,6,8,9,1,30,25,1,12,12,18,0,2,2,25,1,2018,4,39.068913,-77.063227,3146,1,5,2020,1,2,22,10
2,114,5,2,3,3,3935,20,11,13,2,0,7,2,11,9,1,139,10,5,9,9,26,13,4,4,35,0,2016,4,38.993578,-77.032718,36906,0,5,2017,2,6,11,8
3,229,5,1,0,0,3935,20,11,7,3,0,4,1,8,9,2,139,25,2,12,13,18,9,2,2,25,0,2001,5,39.083480,-77.086933,31353,1,5,2019,2,5,10,7
4,795,5,1,3,0,3935,20,11,9,2,0,4,7,8,4,3,139,25,1,9,9,18,17,2,2,40,0,2013,4,38.996942,-77.109168,38537,0,5,2016,3,8,2,17


In [26]:
df=pd.DataFrame(train_scaled_data,columns=col)
df['Fault']=train_data['Fault']
train_data=df

In [27]:
train_data.head(2)

,Local_Case_Number,Agency_Name,ACRS_Report_Type,Route_Type,Cross_Street_Type,Off_Road_Description,Municipality,Related_Non_Motorist,Collision_Type,Weather,Surface_Condition,Light,Traffic_Control,Driver_Substance_Abuse,Non_Motorist_Substance_Abuse,Injury_Severity,Circumstance,Drivers_License_State,Vehicle_Damage_Extent,Vehicle_First_Impact_Location,Vehicle_Second_Impact_Location,Vehicle_Body_Type,Vehicle_Movement,Vehicle_Continuing_Dir,Vehicle_Going_Dir,Speed_Limit,Parked_Vehicle,Vehicle_Year,Equipment_Problems,Latitude,Longitude,Location,report_id,crash_year,crash_quarter,crash_month,crash_day,crash_hour,Fault
0,-0.906966,0.135933,-1.144251,-0.021103,-0.844447,-4.494613,0.344624,0.216333,1.077248,-0.541286,1.44903,0.450872,-1.091975,-0.137463,0.232812,2.409571,-1.186745,-0.28778,-1.396104,0.775011,0.771634,0.326719,-0.306763,0.122028,0.818131,0.607417,-0.198980,0.073209,-0.575623,2.506127,-2.355209,0.383776,0.359767,-0.373083,-0.476245,-0.170255,-1.223804,0.506135,1
1,0.717181,0.135933,-1.144251,-1.017081,-0.844447,0.332039,0.344624,0.216333,1.310307,-0.541286,-0.68082,0.450872,0.247021,-0.137463,0.232812,-0.434935,-3.843196,-0.28778,-0.851477,0.775011,0.771634,0.173171,-1.811965,0.122028,0.126096,-0.675012,5.025635,0.143053,-0.575623,-0.192464,0.503167,-1.446941,0.359767,1.222966,-1.374316,-1.336589,0.721165,-0.640248,1


In [28]:
train_data.isnull().sum().sum()

0


## **Training**

**Splitting train data into train and validation set**

In [29]:
X=train_data.drop(['Fault','Municipality','Local_Case_Number', 'Agency_Name', 'report_id','Vehicle_Year' ],axis=1)
y=train_data['Fault']

In [30]:
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.3, random_state=42)

In [31]:
X_train.columns

Index(['ACRS_Report_Type', 'Route_Type', 'Cross_Street_Type',
       'Off_Road_Description', 'Related_Non_Motorist', 'Collision_Type',
       'Weather', 'Surface_Condition', 'Light', 'Traffic_Control',
       'Driver_Substance_Abuse', 'Non_Motorist_Substance_Abuse',
       'Injury_Severity', 'Circumstance', 'Drivers_License_State',
       'Vehicle_Damage_Extent', 'Vehicle_First_Impact_Location',
       'Vehicle_Second_Impact_Location', 'Vehicle_Body_Type',
       'Vehicle_Movement', 'Vehicle_Continuing_Dir', 'Vehicle_Going_Dir',
       'Speed_Limit', 'Parked_Vehicle', 'Equipment_Problems', 'Latitude',
       'Longitude', 'Location', 'crash_year', 'crash_quarter', 'crash_month',
       'crash_day', 'crash_hour'],
      dtype='object')

**Training Models**

GRADIENT BOOSTING CLASSIFIER

In [32]:
model=GradientBoostingClassifier(max_depth=7, random_state=1,n_estimators=103, max_features=28)
model.fit(X_train,y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=7,
                           max_features=28, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=103,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=1, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [33]:
gbc=model.predict(X_val)
print("Validation Score:")
print(accuracy_score(y_val,gbc))

Validation Score:
0.8523337864957597


XGBOOST CLASSIFIER

In [34]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

xg = XGBClassifier(random_state=1, max_depth=6, n_estimators=163, learning_rate=0.1)
xg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=163, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [35]:
xgbc=xg.predict(X_val)

print("Validation Score:")
print(accuracy_score(y_val,xgbc))

Validation Score:
0.849614811937593


RANDOM FOREST CLASSIFIER

In [36]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=1,max_features=31, max_depth=12,n_estimators=103)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=12, max_features=31,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=103,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [37]:
rfc=clf.predict(X_val)

print("Validation Score:")
print(accuracy_score(y_val,rfc))

Validation Score:
0.8376383763837638


DECISION TREE CLASSIFIER

In [38]:
from sklearn import tree

dtc = tree.DecisionTreeClassifier(random_state=1, max_depth=5)
dtc.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')

In [39]:
x=dtc.predict(X_val)

print("validation score:")
print(accuracy_score(y_val,rfc))

validation score:
0.8376383763837638


##**Testing**

In [40]:
test_data=pd.read_csv("test.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
print(test_data.shape)
print(test_data.columns)
print(f"Total Null values: {test_data.isnull().sum().sum()}")

(77235, 42)
Index(['Id', 'Report Number', 'Local Case Number', 'Agency Name',
       'ACRS Report Type', 'Crash Date/Time', 'Route Type', 'Road Name',
       'Cross-Street Type', 'Cross-Street Name', 'Off-Road Description',
       'Municipality', 'Related Non-Motorist', 'Collision Type', 'Weather',
       'Surface Condition', 'Light', 'Traffic Control',
       'Driver Substance Abuse', 'Non-Motorist Substance Abuse', 'Person ID',
       'Injury Severity', 'Circumstance', 'Drivers License State',
       'Vehicle ID', 'Vehicle Damage Extent', 'Vehicle First Impact Location',
       'Vehicle Second Impact Location', 'Vehicle Body Type',
       'Vehicle Movement', 'Vehicle Continuing Dir', 'Vehicle Going Dir',
       'Speed Limit', 'Driverless Vehicle', 'Parked Vehicle', 'Vehicle Year',
       'Vehicle Make', 'Vehicle Model', 'Equipment Problems', 'Latitude',
       'Longitude', 'Location'],
      dtype='object')
Total Null values: 457654


In [42]:
test_data.columns = test_data.columns.str.replace(' ', '_')
test_data.columns = test_data.columns.str.replace('/', '_')
test_data.columns = test_data.columns.str.replace('-', '_')

In [43]:
test_data['Vehicle_Year']=test_data['Vehicle_Year'].fillna(0)

In [44]:
test_data['report_id'] = test_data['Report_Number'].apply(lambda x: x[0:2])
test_data.report_id.unique()

array(['MP', 'D5', 'E7', 'H2', 'D8'], dtype=object)

In [45]:
test_data = test_data.drop(['Report_Number', 'Person_ID', 'Vehicle_ID', 'Driverless_Vehicle', 
                              'Vehicle_Make', 'Vehicle_Model', 'Road_Name', 'Cross_Street_Name' ], axis=1)
test_data.shape

(77235, 35)

In [46]:
test_data['Crash_Date_Time'] =  pd.to_datetime(test_data['Crash_Date_Time'])
test_data['crash_year'] = test_data['Crash_Date_Time'].dt.year
test_data['crash_quarter'] = test_data['Crash_Date_Time'].dt.quarter
test_data['crash_month'] = test_data['Crash_Date_Time'].dt.month
test_data['crash_day'] = test_data['Crash_Date_Time'].dt.day
test_data['crash_hour'] = test_data['Crash_Date_Time'].dt.hour

In [47]:
test_data=test_data.drop(['Crash_Date_Time'],axis=1)

In [48]:
test_data.Id

0            0
1            1
2            2
3            3
4            4
         ...  
77230    77230
77231    77231
77232    77232
77233    77233
77234    77234
Name: Id, Length: 77235, dtype: int64

In [49]:
l=[]

for i in test_data.columns:
   if test_data[i].dtypes == "object":
     l.append(i)
print(l)
print(len(l))

['Local_Case_Number', 'Agency_Name', 'ACRS_Report_Type', 'Route_Type', 'Cross_Street_Type', 'Off_Road_Description', 'Municipality', 'Related_Non_Motorist', 'Collision_Type', 'Weather', 'Surface_Condition', 'Light', 'Traffic_Control', 'Driver_Substance_Abuse', 'Non_Motorist_Substance_Abuse', 'Injury_Severity', 'Circumstance', 'Drivers_License_State', 'Vehicle_Damage_Extent', 'Vehicle_First_Impact_Location', 'Vehicle_Second_Impact_Location', 'Vehicle_Body_Type', 'Vehicle_Movement', 'Vehicle_Continuing_Dir', 'Vehicle_Going_Dir', 'Parked_Vehicle', 'Equipment_Problems', 'Location', 'report_id']
29


In [50]:
dt=test_data
le = LabelEncoder()

for j in l:
  dt[j]=dt[j].astype('|S')

for i in l:
  dt[i]=le.fit_transform(dt[i])

test_data=dt
test_data.shape

(77235, 39)

In [51]:
test_data.columns

Index(['Id', 'Local_Case_Number', 'Agency_Name', 'ACRS_Report_Type',
       'Route_Type', 'Cross_Street_Type', 'Off_Road_Description',
       'Municipality', 'Related_Non_Motorist', 'Collision_Type', 'Weather',
       'Surface_Condition', 'Light', 'Traffic_Control',
       'Driver_Substance_Abuse', 'Non_Motorist_Substance_Abuse',
       'Injury_Severity', 'Circumstance', 'Drivers_License_State',
       'Vehicle_Damage_Extent', 'Vehicle_First_Impact_Location',
       'Vehicle_Second_Impact_Location', 'Vehicle_Body_Type',
       'Vehicle_Movement', 'Vehicle_Continuing_Dir', 'Vehicle_Going_Dir',
       'Speed_Limit', 'Parked_Vehicle', 'Vehicle_Year', 'Equipment_Problems',
       'Latitude', 'Longitude', 'Location', 'report_id', 'crash_year',
       'crash_quarter', 'crash_month', 'crash_day', 'crash_hour'],
      dtype='object')

In [52]:
col=['Local_Case_Number', 'Agency_Name', 'ACRS_Report_Type',
       'Route_Type', 'Cross_Street_Type', 'Off_Road_Description',
       'Municipality', 'Related_Non_Motorist', 'Collision_Type', 'Weather',
       'Surface_Condition', 'Light', 'Traffic_Control',
       'Driver_Substance_Abuse', 'Non_Motorist_Substance_Abuse',
       'Injury_Severity', 'Circumstance', 'Drivers_License_State',
       'Vehicle_Damage_Extent', 'Vehicle_First_Impact_Location',
       'Vehicle_Second_Impact_Location', 'Vehicle_Body_Type',
       'Vehicle_Movement', 'Vehicle_Continuing_Dir', 'Vehicle_Going_Dir',
       'Speed_Limit', 'Parked_Vehicle', 'Vehicle_Year', 'Equipment_Problems',
       'Latitude', 'Longitude', 'Location', 'report_id', 'crash_year',
       'crash_quarter', 'crash_month', 'crash_day', 'crash_hour']
       
scaler=StandardScaler().fit(test_data[col])
test_scaled_data=scaler.transform(test_data[col])
test_data.head()

,Id,Local_Case_Number,Agency_Name,ACRS_Report_Type,Route_Type,Cross_Street_Type,Off_Road_Description,Municipality,Related_Non_Motorist,Collision_Type,Weather,Surface_Condition,Light,Traffic_Control,Driver_Substance_Abuse,Non_Motorist_Substance_Abuse,Injury_Severity,Circumstance,Drivers_License_State,Vehicle_Damage_Extent,Vehicle_First_Impact_Location,Vehicle_Second_Impact_Location,Vehicle_Body_Type,Vehicle_Movement,Vehicle_Continuing_Dir,Vehicle_Going_Dir,Speed_Limit,Parked_Vehicle,Vehicle_Year,Equipment_Problems,Latitude,Longitude,Location,report_id,crash_year,crash_quarter,crash_month,crash_day,crash_hour
0,0,2816,3,2,0,3,5661,20,9,7,2,0,1,7,0,12,1,148,26,2,1,1,18,7,1,1,25,0,2004,4,38.988440,-77.127668,42836,4,2015,4,12,16,17
1,1,2816,5,2,0,4,5661,8,9,9,6,10,4,7,8,12,1,131,26,5,9,9,18,20,2,4,25,0,2001,4,39.094075,-77.205783,46887,4,2016,2,5,1,19
2,2,277,5,2,3,0,5661,20,9,14,12,0,4,7,8,12,1,148,26,3,13,13,2,9,1,1,35,0,2016,4,39.047180,-77.052688,18958,4,2018,1,3,16,14
3,3,598,5,2,10,10,457,20,9,14,2,11,4,1,11,12,1,148,26,1,12,12,18,0,4,4,25,0,2004,10,39.178007,-77.238475,35278,4,2018,2,5,25,17
4,4,826,5,2,2,3,5661,20,9,7,2,0,4,1,10,12,1,148,73,5,4,4,12,2,1,1,55,0,2008,9,39.056134,-77.154043,24965,4,2017,3,9,23,17


In [53]:
df=pd.DataFrame(test_scaled_data,columns=col)
df['Id']=test_data['Id']
test_data=df

In [54]:
test_data.isnull().sum().sum()

0

In [55]:
test_data.head(5)

,Local_Case_Number,Agency_Name,ACRS_Report_Type,Route_Type,Cross_Street_Type,Off_Road_Description,Municipality,Related_Non_Motorist,Collision_Type,Weather,Surface_Condition,Light,Traffic_Control,Driver_Substance_Abuse,Non_Motorist_Substance_Abuse,Injury_Severity,Circumstance,Drivers_License_State,Vehicle_Damage_Extent,Vehicle_First_Impact_Location,Vehicle_Second_Impact_Location,Vehicle_Body_Type,Vehicle_Movement,Vehicle_Continuing_Dir,Vehicle_Going_Dir,Speed_Limit,Parked_Vehicle,Vehicle_Year,Equipment_Problems,Latitude,Longitude,Location,report_id,crash_year,crash_quarter,crash_month,crash_day,crash_hour,Id
0,1.849806,-1.493458,0.773862,-0.998567,0.024953,0.269280,0.294762,0.115677,-0.829877,-0.519099,-0.651056,-1.636651,0.518325,-4.461830,0.142777,-0.434468,0.365374,-0.285885,-0.286282,-2.047435,-2.053458,0.170493,-0.644738,-0.553385,-0.557945,-0.707207,-0.12568,0.108004,-0.630773,-1.304929,-0.147239,1.054452,0.361506,-1.426205,1.311729,1.569275,0.038047,0.702412,0
1,1.849806,0.129368,0.773862,-0.998567,0.305158,0.269280,-4.155908,0.115677,-0.350102,0.849214,1.468624,0.506914,0.518325,-0.222257,0.142777,-0.434468,-0.441281,-0.285885,1.384362,0.009183,0.003782,0.170493,1.535068,0.145552,1.546988,-0.707207,-0.12568,0.098890,-0.630773,0.154366,-0.925258,1.316834,0.361506,-0.896126,-0.479966,-0.465712,-1.673889,1.084303,1
2,-1.083969,0.129368,0.773862,0.009366,-0.815661,0.269280,0.294762,0.115677,0.849336,2.901684,-0.651056,0.506914,0.518325,-0.222257,0.142777,-0.434468,0.365374,-0.285885,0.270599,1.037492,1.032403,-2.327081,-0.309383,-0.553385,-0.557945,0.197738,-0.12568,0.144458,-0.630773,-0.493465,0.599555,-0.492121,0.361506,0.164030,-1.375813,-1.047137,0.038047,0.129576,2
3,-0.713059,0.129368,0.773862,2.361208,1.986386,-5.264373,0.294762,0.115677,0.849336,-0.519099,1.680592,0.506914,-1.079728,1.367583,0.142777,-0.434468,0.365374,-0.285885,-0.843163,0.780415,0.775248,0.170493,-1.818479,1.543425,1.546988,-0.707207,-0.12568,0.108004,1.661001,1.313841,-1.250864,0.564922,0.361506,0.164030,-0.479966,-0.465712,1.065209,0.702412,3
4,-0.449608,0.129368,0.773862,-0.326612,0.024953,0.269280,0.294762,0.115677,-0.829877,-0.519099,-0.651056,0.506914,-1.079728,0.837636,0.142777,-0.434468,0.365374,3.123395,1.384362,-1.276203,-1.281993,-0.766098,-1.483125,-0.553385,-0.557945,2.007628,-0.12568,0.120155,1.279039,-0.369768,-0.409926,-0.103049,0.361506,-0.366048,0.415881,0.697138,0.836951,0.702412,4


## **Submission**

In [56]:
test=test_data.drop(['Id','report_id', 'Municipality','Local_Case_Number', 'Agency_Name', 'Vehicle_Year'], axis=1)
test.head(5)

,ACRS_Report_Type,Route_Type,Cross_Street_Type,Off_Road_Description,Related_Non_Motorist,Collision_Type,Weather,Surface_Condition,Light,Traffic_Control,Driver_Substance_Abuse,Non_Motorist_Substance_Abuse,Injury_Severity,Circumstance,Drivers_License_State,Vehicle_Damage_Extent,Vehicle_First_Impact_Location,Vehicle_Second_Impact_Location,Vehicle_Body_Type,Vehicle_Movement,Vehicle_Continuing_Dir,Vehicle_Going_Dir,Speed_Limit,Parked_Vehicle,Equipment_Problems,Latitude,Longitude,Location,crash_year,crash_quarter,crash_month,crash_day,crash_hour
0,0.773862,-0.998567,0.024953,0.269280,0.115677,-0.829877,-0.519099,-0.651056,-1.636651,0.518325,-4.461830,0.142777,-0.434468,0.365374,-0.285885,-0.286282,-2.047435,-2.053458,0.170493,-0.644738,-0.553385,-0.557945,-0.707207,-0.12568,-0.630773,-1.304929,-0.147239,1.054452,-1.426205,1.311729,1.569275,0.038047,0.702412
1,0.773862,-0.998567,0.305158,0.269280,0.115677,-0.350102,0.849214,1.468624,0.506914,0.518325,-0.222257,0.142777,-0.434468,-0.441281,-0.285885,1.384362,0.009183,0.003782,0.170493,1.535068,0.145552,1.546988,-0.707207,-0.12568,-0.630773,0.154366,-0.925258,1.316834,-0.896126,-0.479966,-0.465712,-1.673889,1.084303
2,0.773862,0.009366,-0.815661,0.269280,0.115677,0.849336,2.901684,-0.651056,0.506914,0.518325,-0.222257,0.142777,-0.434468,0.365374,-0.285885,0.270599,1.037492,1.032403,-2.327081,-0.309383,-0.553385,-0.557945,0.197738,-0.12568,-0.630773,-0.493465,0.599555,-0.492121,0.164030,-1.375813,-1.047137,0.038047,0.129576
3,0.773862,2.361208,1.986386,-5.264373,0.115677,0.849336,-0.519099,1.680592,0.506914,-1.079728,1.367583,0.142777,-0.434468,0.365374,-0.285885,-0.843163,0.780415,0.775248,0.170493,-1.818479,1.543425,1.546988,-0.707207,-0.12568,1.661001,1.313841,-1.250864,0.564922,0.164030,-0.479966,-0.465712,1.065209,0.702412
4,0.773862,-0.326612,0.024953,0.269280,0.115677,-0.829877,-0.519099,-0.651056,0.506914,-1.079728,0.837636,0.142777,-0.434468,0.365374,3.123395,1.384362,-1.276203,-1.281993,-0.766098,-1.483125,-0.553385,-0.557945,2.007628,-0.12568,1.279039,-0.369768,-0.409926,-0.103049,-0.366048,0.415881,0.697138,0.836951,0.702412


In [57]:
test.columns

Index(['ACRS_Report_Type', 'Route_Type', 'Cross_Street_Type',
       'Off_Road_Description', 'Related_Non_Motorist', 'Collision_Type',
       'Weather', 'Surface_Condition', 'Light', 'Traffic_Control',
       'Driver_Substance_Abuse', 'Non_Motorist_Substance_Abuse',
       'Injury_Severity', 'Circumstance', 'Drivers_License_State',
       'Vehicle_Damage_Extent', 'Vehicle_First_Impact_Location',
       'Vehicle_Second_Impact_Location', 'Vehicle_Body_Type',
       'Vehicle_Movement', 'Vehicle_Continuing_Dir', 'Vehicle_Going_Dir',
       'Speed_Limit', 'Parked_Vehicle', 'Equipment_Problems', 'Latitude',
       'Longitude', 'Location', 'crash_year', 'crash_quarter', 'crash_month',
       'crash_day', 'crash_hour'],
      dtype='object')

In [58]:
# model- gradient boosting classifier
# xg- xgboost
pred=model.predict(test)

In [59]:
print(pred[:10])

[1 0 0 1 1 1 1 0 0 1]


In [60]:
test_data['Fault']=pred
test_data.head(5)

,Local_Case_Number,Agency_Name,ACRS_Report_Type,Route_Type,Cross_Street_Type,Off_Road_Description,Municipality,Related_Non_Motorist,Collision_Type,Weather,Surface_Condition,Light,Traffic_Control,Driver_Substance_Abuse,Non_Motorist_Substance_Abuse,Injury_Severity,Circumstance,Drivers_License_State,Vehicle_Damage_Extent,Vehicle_First_Impact_Location,Vehicle_Second_Impact_Location,Vehicle_Body_Type,Vehicle_Movement,Vehicle_Continuing_Dir,Vehicle_Going_Dir,Speed_Limit,Parked_Vehicle,Vehicle_Year,Equipment_Problems,Latitude,Longitude,Location,report_id,crash_year,crash_quarter,crash_month,crash_day,crash_hour,Id,Fault
0,1.849806,-1.493458,0.773862,-0.998567,0.024953,0.269280,0.294762,0.115677,-0.829877,-0.519099,-0.651056,-1.636651,0.518325,-4.461830,0.142777,-0.434468,0.365374,-0.285885,-0.286282,-2.047435,-2.053458,0.170493,-0.644738,-0.553385,-0.557945,-0.707207,-0.12568,0.108004,-0.630773,-1.304929,-0.147239,1.054452,0.361506,-1.426205,1.311729,1.569275,0.038047,0.702412,0,1
1,1.849806,0.129368,0.773862,-0.998567,0.305158,0.269280,-4.155908,0.115677,-0.350102,0.849214,1.468624,0.506914,0.518325,-0.222257,0.142777,-0.434468,-0.441281,-0.285885,1.384362,0.009183,0.003782,0.170493,1.535068,0.145552,1.546988,-0.707207,-0.12568,0.098890,-0.630773,0.154366,-0.925258,1.316834,0.361506,-0.896126,-0.479966,-0.465712,-1.673889,1.084303,1,0
2,-1.083969,0.129368,0.773862,0.009366,-0.815661,0.269280,0.294762,0.115677,0.849336,2.901684,-0.651056,0.506914,0.518325,-0.222257,0.142777,-0.434468,0.365374,-0.285885,0.270599,1.037492,1.032403,-2.327081,-0.309383,-0.553385,-0.557945,0.197738,-0.12568,0.144458,-0.630773,-0.493465,0.599555,-0.492121,0.361506,0.164030,-1.375813,-1.047137,0.038047,0.129576,2,0
3,-0.713059,0.129368,0.773862,2.361208,1.986386,-5.264373,0.294762,0.115677,0.849336,-0.519099,1.680592,0.506914,-1.079728,1.367583,0.142777,-0.434468,0.365374,-0.285885,-0.843163,0.780415,0.775248,0.170493,-1.818479,1.543425,1.546988,-0.707207,-0.12568,0.108004,1.661001,1.313841,-1.250864,0.564922,0.361506,0.164030,-0.479966,-0.465712,1.065209,0.702412,3,1
4,-0.449608,0.129368,0.773862,-0.326612,0.024953,0.269280,0.294762,0.115677,-0.829877,-0.519099,-0.651056,0.506914,-1.079728,0.837636,0.142777,-0.434468,0.365374,3.123395,1.384362,-1.276203,-1.281993,-0.766098,-1.483125,-0.553385,-0.557945,2.007628,-0.12568,0.120155,1.279039,-0.369768,-0.409926,-0.103049,0.361506,-0.366048,0.415881,0.697138,0.836951,0.702412,4,1


In [61]:
result=test_data[['Id','Fault']]
result.to_csv("submission", index=False)